In [ ]:
!pip install docarray
#!pip install sentence-transformers

## Retrievers en LangChain

##### Funcionamiento de as_retriever

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = DocArrayInMemorySearch.from_texts(

    ["River Plate es un equipo de futbol", "A Juan le gusta leer"],
    embedding=embeddings
)
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("Que es River Plate?")
retriever.get_relevant_documents("Que le gusta a Juan?")


#### Convertir un vector store a Retriever
VectorStoreBackedRetriever

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

loader = PyPDFLoader("./directorio/Estructuras.pdf")
documents = loader.load()
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")    #"sentence-transformers/all-mpnet-base-v2"

# Se crea el vector store
persist_directory = "chroma_db"
db = Chroma.from_documents(documents=texts, embedding=embeddings, persist_directory=persist_directory
                          )

In [ ]:
# Definir el numero de chunks a devolver
retriever = db.as_retriever(search_kwargs={"k": 3}, search_type="similarity") #mmr
retriever.invoke("Como crear una lista")

In [ ]:
# Definir el numero de chunks a devolver
retriever = db.as_retriever(search_kwargs={"k": 2}, search_type="mmr")
retriever.invoke("derechos")

##### Imponer un umbral de puntaje

In [ ]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.1}
)
retriever.invoke("derechos")

### ParentDocumentRetriever

#### Preparación

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

loader = TextLoader("directorio/Un mundo feliz - ALDOUS HUXLEY.txt",       # 447000 caracteres
               encoding="utf-8")

docs = loader.load()
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#### Recuperar documentos de origen

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Se crea una BD vectorial
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=embeddings,
)

# Se particionan los chunks menores
menor_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)

# En store se guardan los documentos de origen
store = InMemoryStore()

# Se instancia el retriever
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=menor_splitter,
)
# Agregar los documentos al retriever
retriever.add_documents(docs)
# Devuelve las claves de los documentos cargados
list(store.yield_keys())

In [ ]:
sub_docs = vectorstore.similarity_search("Como es el sistema de gobierno?")
print(sub_docs[0].page_content)
print(len(sub_docs))

In [ ]:
retrieved_docs = retriever.invoke("Como es el sistema de gobierno?")
print(len(retrieved_docs[0].page_content))

### Recuperar los chunks mayores pero no los documentos
Hacer particiones mayores

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Instanciar el particionador mayor
mayor_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

# Instanciar el particionador menor
menor_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)

# Crear el espacio vectorial donde almacenar los chunks menores
# Estos chunks son incrustados
vectorstore = Chroma(
     collection_name="split_parents", 
     embedding_function=embeddings
)

# Almacen para los chunks mayores en memoria
# Estos chunks no son incrustados
store = InMemoryStore()

# Se instancia el retriever, que hara la mayor parte del trabajo
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=menor_splitter,
    parent_splitter=mayor_splitter,
)

In [4]:
# Se agregan los documentos
# Tareas:
# 1 Agregar los documentos originales
# 2 Particionar en chunks mayores y asignar un ID a cada uno
# 3 Almacenar los chunks mayores en docstore, sin incrustar
# 4 Particionar los chunks mayores en chunks menores
# 5 Incrustarlos y almacenarlos en el especio vectorial
# 6 Asignar a los chunks menores los IDs de los mayores para mantener la conexion
retriever.add_documents(docs)

In [5]:
num_chunks_mayores = len(retriever.docstore.store.items())
num_chunks_menores = len(set(retriever.vectorstore.get()['documents']))
# Con set() se eliminan chunks duplicados
print (f"Hay {num_chunks_mayores} chunks mayores y {num_chunks_menores} chunks menores.")

Hay 1160 chunks mayores y 1693 chunks menores.


In [10]:
# Recuperar los chunks menores mas relevantes
sub_docs = vectorstore.similarity_search("Como es el sistema de gobierno?")
print(f'Se recuperaron {len(sub_docs)} chunks menores relevantes.')
print(f'Este chunk tiene {len(sub_docs[0].page_content)} caracteres.')
print(f'El texto es: {sub_docs[0].page_content}')
# El indice identifica el chunk mayor fuente

Se recuperaron 4 chunks menores relevantes.
Este chunk tiene 370 caracteres.
El texto es: Desde luego, no hay razón alguna para que el nuevo totalita- 
rismo se parezca al antiguo. El Gobierno, por medio de porras 
y piquetes de ejecución, hambre artificialmente provocada, en- 
carcelamientos en masa y deportación también en masa no es 
solamente inhumano (a nadie, hoy día, le importa demasiado 
este hecho); se ha comprobado que es ineficaz, y en una época


In [12]:
# Cuando se invoca el retriever:
# 1 Recupera los chunks menores mas relevantes mediante una busqueda por similitud semantica
# 2 Recupera los chunks mayores en docstore
# 3 Guarda en retrieved_docs los chunks mayores que son fuente de los menores mas relevantes
# 4 Construye el contexto a partir de estos.
retrieved_docs = retriever.invoke("Como es el sistema de gobierno?")
print(len(retrieved_docs))
print(retrieved_docs[0].metadata)
len(retrieved_docs[2].page_content)

4
{'source': 'directorio/Un mundo feliz - ALDOUS HUXLEY.txt'}


852

##### Invocar el modelo

In [13]:
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
# Se instancia un modelo de lenguaje
llm = ChatGroq(model="llama3-8b-8192")
# Se construye el prompt
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Responda en español.

{context}

Question: {question}
Answer:"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

question = "Como es el sistema de gobierno?"
mensaje = prompt.format_prompt(context=retrieved_docs, question=question)
res = llm.invoke(input=mensaje)
res.content

'Según el texto, el sistema de gobierno en este mundo no es el tradicional y violento, donde se utilizan la coerción y la represión para mantener el orden. En su lugar, el sistema de gobierno es más avanzado y utiliza la tecnología para condicionar a la población de manera que no puedan hacer otra cosa más que lo que deben hacer.'